# Пакеты и файлы

**Пакеты** в Go являются аналогом библиотек или модулей в других языках и вносит в программу модульность, инкапсуляцию, раздельную компиляцию и усиливает повторное использование кода. Код пакета располагается в одном или нескольких **.go** файлах, которые в свою очередь находятся в папке, название которой обычно совпадает с названием пакета. Каждый пакет формирует отдельное **пространство имён**(name space) для находящихся внутри него объявлений. Пакет также скрывает ненужную для его использования информацию от клиента(инкапсуляция): объявления, которые начинаются с большой буквы **экспортируются** из пакета; остальные объявления касаются только внутренних механизмов работы пакета и не нужны клиенту.

Для примера рассмотрим собственный пакет под названием "fish". Этот пакет находится в папке fish, которая находится в том же месте, что и этот блокнот. Пакет "fish" состоит из трёх файлов: kind.go, desc.go, fish.go. Код в этих файлах представлен ниже:

### kind.go
Файл kind.go содержит описание видов рыб. Т.к. тип FishKind начинается с большой буквы, то он экспортируется, т.е. клиенты могут создать переменные такого типа.

In [ ]:
package fish

type FishKind int

const (
	Sea        = 0
	FreshWater = 1
	Unknown    = 2
)

### desc.go
Файл desc.go содержит описание различных видов рыб. В этом файле и тип description и словарь fishDesc начинаются с маленькой буквы, поэтому эти объявления не экпортируются, но к ним можно обращаться из других файлов этого же пакета как это делается в файле fish.go.

In [ ]:
package fish

type description struct {
	kind       FishKind
	mainColor  string
	endangered bool
}

var fishDesc map[string]description = map[string]description{
	"Salmon": {Sea, "grey, white", false},
	"Trout":  {FreshWater, "green, white, red", false},
}


### fish.go
Файл fish.go содержит только одну экспортируемую функцию, которая возвращает вид рыбы по её имени. Т.к. словарь fishDesc находится в том же пакете, что и этот файл, к этому словарю можно обращаться даже несмотря на то, что он начинается с маленькой буквы.

In [ ]:
package fish

func Kind(fishName string) FishKind {
	desc, ok := fishDesc[fishName]
	if ok {
		return desc.kind
	}
	return Unknown
}

Этот пакет мог бы быть использован, например, следующим образом:

In [ ]:
package main

import (
	"fish"
	"fmt"
)

func main() {
	if fish.Kind("Salmon") == fish.Sea {
		fmt.Println("Salmon is sea fish.")
	}
}

Чтобы этот код работал, необходимо папку fish скопировать в $GOPATH/src.

## Импортирование

Для импорта пакета используется объявление **import**, после которого указывается путь до импортируемого пакета относительно предопределённых папок в системе. Одной из таких папок является папка $GOPATH/src, которая настраивается при первоначальной установке Go в системе. Другой папкой является GOROOT/src(узнать чему равен GOROOT можно, запустив команду **go env**).

## Инициализация пакета

Инициализация пакета начинается с инициализации переменных уровня пакета в том порядке, в котором они объявлены. При этом есть исключение, что зависимости разрешаются первыми. Например, в коде приведённом ниже переменная  fullName инициализируется в третью очередь, т.к. у неё есть зависимости от других переменных. В первую очередь инициализируется firstName, а за ней secondName.

In [ ]:
package main

import "fmt"

var fullName = firstName + " " + secondName
var firstName = "John"
var secondName = "Smith"

func main() {
	fmt.Println(fullName)
}


Если пакет состоит из нескольких .go файлов они инициализируется в том порядке, в котором они **подаются** на вход компилятору. Команда go **сортирует** .go файлы по имени перед запуском компилятора.
Когда необходимо инициализировать более сложные переменные типа таблиц можно воспользоваться специальной функцией под названием **init**. Любой файл может содержать произвольное количество функций следующего вида:

func init() { ... }

Эту функцию нельзя вызывать в явном виде, но во всём остальном она ничем не отличается от обычных функций. При запуске программы вызываются все функции init в порядке, в котором они объявлены внутри файла. Это действие выполняется для каждого файла.

Пакеты инициализируются по очереди в том порядке, в котором они импортируются. При этом сначала инициализируются зависимости: инициализация продвигается снизу вверх, пакет main инициализируется в последнюю очередь.